In [5]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(".env")

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
)

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL"),
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, many other Azure AI services support **Customer Managed Keys (CMK)** to help organizations maintain control over their data by managing encryption keys. This feature is part of Microsoft’s effort to ensure security and compliance for data-sensitive workloads. Below are examples of Azure AI services that support customer-managed keys:

### 1. **Azure Cognitive Search**
   - Azure Cognitive Search allows you to use customer-managed keys for encrypting your data at rest. This lets you maintain full control over the encryption keys stored in Azure Key Vault or Azure Managed HSM.

### 2. **Azure Machine Learning**
   - Azure Machine Learning supports customer-managed keys for encrypting both the **workspace's default storage account** and **data in Azure Machine Learning resources**, including datasets, models, and experiments.

### 3. **Azure Cognitive Services**
   - Many Cognitive Services (e.g., Text Analytics, Translator, and Form Recognizer) allow customers to use CMK to encrypt 

In [ ]:
from neo4jDb import get_driver

def get_chat_history(driver=get_driver(),session_id=None,database=None) :
    """
    Extrae el historial de conversación desde Neo4j.

    :param driver: Instancia del driver de Neo4j.
    :param session_id: ID de la sesión de chat.
    :param database: Base de datos a utilizar (por defecto "neo4j").
    :return: Lista de tuplas con (tipo de mensaje, contenido).
    """
    query = """
    MATCH (s:Session)-[:LAST_MESSAGE]->(last:Message)
    WHERE s.id = $session_id
    MATCH p = (last)<-[:NEXT*]-(msg:Message)
    UNWIND nodes(p) as msgs
    RETURN DISTINCT msgs.type, msgs.content
    """

    # Ejecutar la consulta
    with driver.session(database=database) as session:
        result = session.run(query, session_id=session_id)
        
        # Mostrar los resultados
        for record in result:
            print(f"Type: {record['msgs.type']}, Content: {record['msgs.content']}")
    driver.close()

    return None

# Ejemplo de uso:
get_chat_history(session_id="bb198b2b", database="u1234568")

Type: ai, Content: Entiendo que puede ser frustrante no obtener la información que necesitas de inmediato. Si tienes acceso a los responsables del proyecto o a los documentos oficiales, seguramente encontrarás lo que buscas. Si necesitas orientación sobre cómo buscar esa información o tienes alguna otra consulta, no dudes en pedírmelo. ¡Estoy aquí para ayudarte!
Type: human, Content: bueno :(
Type: ai, Content: Lo siento, pero no tengo acceso a información específica y detallada sobre los requisitos técnicos del Tren Maya, en particular para el tramo 5, ni sobre las cláusulas específicas que establece la Marina en relación con este proyecto. Te recomendaría consultar los documentos oficiales del proyecto, como licitaciones, especificaciones técnicas o acuerdos con la Secretaría de Marina, para obtener información precisa y actualizada. También puedes contactar directamente a las autoridades responsables del Tren Maya o a las entidades involucradas en el proyecto.
Type: human, Content: 

In [1]:
import neo4jDb
from chat import Chat

# Se crea una sesion para el chat o se pasa uno ya existente
DB_ID = "u1234568" # incializar con un minuscul

# Se inicializa el chat
chat_test = Chat(DB_ID)
chat_test.get_chat_history(session_id="ejemplo123") # Se obtiene el historial de la conversacion

Type: ai, Content: Me comentaste anteriormente que eres Alejandro y que eres becario en Sener. ¿Hay algo más que te gustaría compartir o en lo que pueda ayudarte?
Type: human, Content: quien soy?
Type: ai, Content: ¡Hasta luego, Alejandro! Mucho éxito en tu experiencia como becario en Sener. Si necesitas ayuda en el futuro, aquí estaré para apoyarte. ¡Que tengas un excelente día!
Type: human, Content: bueno adios
Type: ai, Content: ¡Claro que sí! Sener es una empresa multinacional de ingeniería y tecnología con una sólida trayectoria en diversos sectores, como energía, infraestructuras, aeroespacial y naval. En México, Sener ha participado en proyectos emblemáticos, como el Tren Maya, plantas de energía y otras iniciativas de gran impacto. 

Como becario, seguramente tendrás la oportunidad de aprender y contribuir en proyectos importantes. Si necesitas información específica sobre algún tema o proyecto dentro de Sener, no dudes en pedírmelo, y con gusto te ayudaré. ¡Bienvenido a la exp